In [ ]:
import numpy as np
import cv2
import imutils
import os
import math
%matplotlib inline
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files

In [ ]:
class preProcessing:
    def __init__(self):
      self.img = None
      self.shape = None
      self.len_quad = 150 
      self.kernel = np.ones((3,3),np.uint8)  
      # self.kernel=np.array([[0, 1, 0],
      #                      [1, 1, 1],
      #                      [0, 1, 0]])
      # self.kernel = (5,5)  

    def resize(self, img):
        ratio = len(img[0]) / 1000.0
        self.img = cv2.resize(img, (0,0), fx=1/ratio, fy=1/ratio)
        
        self.shape = self.img.shape
        # print("resize", self.shape)
        return self.img

    def gray2binary(self, img):
        return (200 < img) & (img <= 100)

    def rgb2gray(self, img):
        r,g,b = img[:,:,0],img[:,:,1],img[:,:,2]
        img = 0.2989 * r + 0.5870 * g + 0.1140 * b
        return img

    def gray(self, img):
        # cv2_imshow(img)
        #gray and filter the image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return img

    def bilateralFilter(self, img):
        #bilateral filtering removes noise and preserves edges
        img = cv2.bilateralFilter(img, 11, 17, 17)
        # cv2_imshow(img)
        return img


    def padding(self, img):
        size = self.kernel.shape[1]
        padSize = size//2
        rows = img.shape[0]
        columns = img.shape[1]
        padImg = np.zeros((rows+2*padSize, columns+2*padSize), dtype=np.uint8)
        # Slicing
        padImg[padSize:rows+padSize, padSize:columns+padSize] = img
        return padImg

    def erode(self, img):
        size = self.kernel.shape[1]
        padImg = self.padding(img)
        rows = padImg.shape[0]
        columns = padImg.shape[1]
        output = np.zeros((rows, columns), dtype=np.uint8)
        for i in range(0, rows):
            for j in range(0, columns):
                # Slicing
                portion = padImg[i:i+size, j:j+size]
                # sum of Kernel and window
                portion1 = portion.flatten()
                portion2 = self.kernel.flatten()
                p1 = (np.sum(portion1))
                p2 = (np.sum(portion2))*255
                # if Fit Condition Satisfies
                if p1 == p2:
                    output[i, j] = 255
                else:
                    output[i, j] = np.min(portion1)
        return output

    def dilate(self, img):
        size = self.kernel.shape[1]
        padImg = self.padding(img)
        rows = padImg.shape[0]
        columns = padImg.shape[1]
        output = np.zeros((rows, columns), dtype=np.uint8)
        for i in range(0, rows):
            for j in range(0, columns):
                # Slicing
                portion = padImg[i:i+size, j:j+size]
                portion1 = portion.flatten()
                # if Hit Condition Satisfies
                if 255 in portion1:
                    output[i, j] = 255
                else:
                    output[i, j] = np.max(portion1)
        return output

    def opening(self, img):
        # First apply Erosion
        erosion = self.erosde(img)
        # secondly apply Dilation on Eroded
        output = self.dilation(erosion)
        return output

    def closing(self, img):
        # First apply Dilation
        dilation = self.dilation(img)
        # secondly apply Erosion on Dilated
        output = self.erode(dilation)
        return output

    def thresholding(self, f, L):
        # create a new image with zeros
        f_tr = np.ones(f.shape).astype(np.uint8)
        # setting to 0 the pixels below the threshold
        f_tr[np.where(f < L)] = 0
        return f_tr

    def otsu_threshold(self, img, max_L):        
        M = np.product(img.shape)
        min_var = []
        hist_t,_ = np.histogram(img, bins=256, range=(0,256))
        
        img_t = self.thresholding(img, 0)
        
        for L in np.arange(1, max_L):
            img_ti = self.thresholding(img, L)
            # computing weights
            w_a = np.sum(hist_t[:L])/float(M)
            w_b = np.sum(hist_t[L:])/float(M)
            # computing variances
            sig_a = np.var(img[np.where(img_ti == 0)])
            sig_b = np.var(img[np.where(img_ti == 1)])
            
            min_var = min_var + [w_a*sig_a + w_b*sig_b]
            
        img_t = self.thresholding(img, np.argmin(min_var))
        
        return img_t
        
    def otsu(self, img):
        # OTSU
        ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
        # cv2_imshow( img)
        return img

    

In [ ]:
## Dictionary with imagery info
img_metadata = {'1655673922574.jpg':[['2','2','57'],['B','C','A'],['0013658247']],
                '1655673922606.jpg':[['2','2','57'],['B','C','A'],['0013658247']],
                '1655673922651.jpg':[['1','2','59'],['B','D','B'],['6420246864']],
                '1655673922674.jpg':[['3','2','55'],['B','C','D'],['1023456789']],
                '1655673922745.jpg':[['2','2','57'],['B','C','A'],['0013658247']],
                'prova_standard_1_page-0001.jpg':[['1','2','59'],['D','A','B'],['11933342']],
                'prova_standard_2_page-0001.jpg':[['3','2','55'],['B','A','C'],['62699574']],
                'prova_standard_3_page-0001.jpg':[['2','2','57'],['C','A','D'],['24570266']],
                'prova_standard_1_azul.jpg':[['3','2','55'],['B','C','A'],['94573802']],
                'prova_standard_2_azul.jpg':[['2','2','57'],['B','C','A'],['14913842']],
                'prova_standard_3_azul.jpg':[['1','2','59'],['B','C','A'],['00933342']],
                'prova_noise_1.jpg':[['3','2','55'],['B','C','A'],['94573802']],
                'prova_noise_2.jpg':[['2','2','57'],['B','C','A'],['14913842']],
                'prova_noise_3.jpg':[['1','2','59'],['B','C','A'],['00933342']],
                'prova_noise_4.jpg':[['2','2','57'],['B','C','A'],['14913842']],
                'prova_noise_5.jpg':[['1','2','59'],['D','A','B'],['11933342']],
                'prova_noise_6.jpg':[['3','2','55'],['B','C','A'],['94573802']],
                'prova_noise_7.jpg':[['2','2','57'],['B','C','A'],['14913842']],
                'prova_noise_8.jpg':[['1','2','59'],['D','A','B'],['11933342']],}

In [1]:

pre = preProcessing()
blob = findBlob()

image = pre.rgb2gray(np.array(cv2.imread('exames/1655673922674.jpg')))


cv2_imshow(image)

print("shape", image.shape)
# # image_original = pre.resize(image)
# # cv2_imshow(image_original)
# # Operacoes morfologicas
# # image_original = pre.gray(image_original)
# # image_original = pre.bilateralFilter(image_original)

# # image = pre.closing(image_original).copy()
# cv2_imshow(image)

image = pre.dilate(image)
cv2_imshow(image)

print("shape", image.shape)
image = pre.opening(image)
cv2_imshow(image)


# im = gray2binary(rgb2gray(np.array(Image.open('morphological.png'))))
# im_dilation=convolution_dilation(structuring_element,im)
# im_erosion=convolution_erosion(structuring_element,im)
# im_ot=convolution_erosion(structuring_element,convolution_dilation(structuring_element,im))

NameError: ignored